In [1]:
import sys
import os

project_dir = os.path.abspath(os.path.join('..'))
if project_dir not in sys.path:
    sys.path.append(project_dir)

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from src.data_loader import DenoisingDataset, get_dataloader
from src.models import SimpleCNN, DeeperCNN
from src.evaluate import mse, psnr, mae



In [2]:
train_low_dir = '../data/train/low'
train_high_dir = '../data/train/high'
val_low_dir = '../data/val/low'
val_high_dir = '../data/val/high'

train_loader = get_dataloader(train_low_dir, train_high_dir, batch_size=16, shuffle=True)
val_loader = get_dataloader(val_low_dir, val_high_dir, batch_size=16, shuffle=False)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/val/low'

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, padding=1)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class DeeperCNN(nn.Module):
    def __init__(self):
        super(DeeperCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, padding=1)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


: 

In [ ]:
#training function

def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for low_img, high_img in train_loader:
            low_img, high_img = low_img.to(device), high_img.to(device)
            
            optimizer.zero_grad()
            outputs = model(low_img)
            loss = criterion(outputs, high_img)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * low_img.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}')
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for low_img, high_img in val_loader:
                low_img, high_img = low_img.to(device), high_img.to(device)
                outputs = model(low_img)
                loss = criterion(outputs, high_img)
                val_loss += loss.item() * low_img.size(0)
        
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}')
    
    return model, train_losses, val_losses


: 

In [ ]:
def evaluate_model(model, val_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    
    total_mse, total_psnr, total_mae = 0, 0, 0
    with torch.no_grad():
        for low_img, high_img in val_loader:
            low_img, high_img = low_img.to(device), high_img.to(device)
            outputs = model(low_img)
            outputs = outputs.cpu().numpy()
            high_img = high_img.cpu().numpy()
            
            for i in range(len(outputs)):
                total_mse += mse(outputs[i], high_img[i])
                total_psnr += psnr(outputs[i], high_img[i])
                total_mae += mae(outputs[i], high_img[i])
    
    num_images = len(val_loader.dataset)
    avg_mse = total_mse / num_images
    avg_psnr = total_psnr / num_images
    avg_mae = total_mae / num_images
    print(f'MSE: {avg_mse:.4f}, PSNR: {avg_psnr:.4f}, MAE: {avg_mae:.4f}')
    return avg_mse, avg_psnr, avg_mae


: 

In [ ]:
simple_cnn = SimpleCNN()
trained_simple_cnn, train_losses, val_losses = train_model(simple_cnn, train_loader, val_loader)
simple_cnn_metrics = evaluate_model(trained_simple_cnn, val_loader)


deeper_cnn = DeeperCNN()
trained_deeper_cnn, train_losses, val_losses = train_model(deeper_cnn, train_loader, val_loader)
deeper_cnn_metrics = evaluate_model(trained_deeper_cnn, val_loader)

print("SimpleCNN Metrics: ", simple_cnn_metrics)
print("DeeperCNN Metrics: ", deeper_cnn_metrics)


: 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses)+1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_losses)+1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


: 

: 